In [1]:
__depends__=[]
__dest__="../results/hydrad_varying_tau_results.pickle"

# Read HYDRAD Results
In our paper, we make various comparisons between EBTEL and the field-aligned code HYDRAD. However, runs of HYDRAD are computationally expensive and it is not feasible to do these on the fly. We are in the process of making HYDRAD available to the public so at some point a more _open_ solution may be possible.

For now, we provide only the output of HYDRAD which will be averaged over the coronal portion of the loop to extract timeseries for the electron and ion temperatures as well as density.

Unfortunately, for the entire parameter space covered, the dataset is quite large. Here we provide the utilities for doing the time averaging, but do not as of yet have a way of efficiently and openly distributing this dataset. If you know of a viable solution, do not hesitate to contact us. We would be happy to provide this dataset to anyone interested.

For now, we include only the already-time-averaged results. We merely read in these text files and convert them to Python pickle files. If you have the full HYDRAD dataset in `../results/static/`, the averaging calculation will be performed.

In [2]:
import os
import pickle

import numpy as np

Compute the coronal averages for temperature and density over the whole parameter space: electron heating, ion heating, single fluid and $\tau=20,40,200,500$ s.

First, set some options and define the range of parameters.

In [3]:
hfRes_format = '../results/static/HYDRAD_raw/%s/HYDRAD_%d/profile%d.phy'
hydrad_labs = [20,40,200,500]
hydrad_res = {'electron':{},'ion':{},'single':{},
              'loop_midpoint':4.5e+9, 'time':np.arange(0,5001)}
int_perc = 0.8

Define a function to do the spatial averaging.

In [4]:
def spatial_average(s,f,mp,eps_mp):
    #calculate bounds
    mp_lower = mp - eps_mp/2.0*mp
    mp_upper = mp + eps_mp/2.0*mp
    #find f and s within specified bounds
    i_eb = np.where((s>=mp_lower) & (s<=mp_upper))[0]
    s_eb = s[i_eb]
    f_eb = f[i_eb]
    #take average
    delta_s = np.gradient(s_eb)
    return np.average(f_eb,weights=delta_s)

Now, if the raw HYDRAD results are in the appropriate directory, do the time average over all of them and save them to a data structure. Otherwise, just load the time-dependent spatial averages from a binary blob.

In [5]:
if os.path.isdir('../results/static/HYDRAD_raw') and not os.path.isfile('../results/static/hydrad_varying_tau_results.pickle'):
    for key in hydrad_res:
        if key=='loop_midpoint' or key=='time':
            continue
        for hl in hydrad_labs:
            Te_avg = []
            Ti_avg = []
            n_avg = []
            for t in hydrad_res['time']:
                #Load results
                temp = np.loadtxt(hfRes_format%(key,hl,t))
                #slice
                s_temp = temp[:,0]
                Te_temp = temp[:,7]
                Ti_temp = temp[:,8]
                n_temp = temp[:,3]
                #save averages
                Te_avg.append(spatial_average(s_temp,Te_temp,hydrad_res['loop_midpoint'],int_perc))
                Ti_avg.append(spatial_average(s_temp,Ti_temp,hydrad_res['loop_midpoint'],int_perc))
                n_avg.append(spatial_average(s_temp,n_temp,hydrad_res['loop_midpoint'],int_perc))

            hydrad_res[key]['tau%ds'%hl] = {'Te':Te_avg,'Ti':Ti_avg,'n':n_avg}
else:
    with open('../results/static/hydrad_varying_tau_results.pickle','rb') as f:
        hydrad_res = pickle.load(f)

Finally, save the results as a serialized pickle file.

In [6]:
with open(__dest__,'wb') as f:
    pickle.dump(hydrad_res,f)